In [58]:

from key import KEY as OPENAI_API_KEY
from key import SS_KEY as SS_API_KEY
from urllib.request import Request, urlopen

import json
import os
import pickle
import time
import requests
from bs4 import BeautifulSoup

# With Semantic Scholar

In [ ]:
# Directly define the API key (Reminder: Securely handle API keys in production environments)
api_key = SS_API_KEY

# Define headers with API key
headers = {'x-api-key': api_key}

In [67]:
def get_paper_data(paper_id):
  url = 'https://api.semanticscholar.org/graph/v1/paper/' + paper_id

  # Define which details about the paper you would like to receive in the response
  paper_data_query_params = {'fields': 'title,isOpenAccess,openAccessPdf'}

  # Send the API request and store the response in a variable
  response = requests.get(url, params=paper_data_query_params)
  if response.status_code == 200:
    return response.json()
  else:
    return None

# def download_file(url, filename):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Check if the request was successful
#         with open(filename, 'wb') as file:
#             file.write(response.content)
#         print(f"File downloaded successfully: {filename}")
#     except requests.exceptions.RequestException as e:
#         print(f"Error occurred: {e}")
#         print("no gusto")
  

def download_file(url, filename):
  try:
    response = requests.get(url=url, headers={'User-Agent': 'Mozilla/5.0'})

    with open(filename, 'wb') as file:
      file.write(response.content)
    print(f"File downloaded successfully: {filename}")
  
  except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")




In [53]:
# Define the API endpoint URL
url = 'https://api.semanticscholar.org/graph/v1/paper/search'

# More specific query parameter
query_params = {'query': 'Mapping the evolution of T cell states during response and resistance to adoptive cellular therapy', }

# Send the API request
search_response = requests.get(url, params=query_params, headers=headers)

# Check if the request was successful (status code 200)
if search_response.status_code == 200:
  search_response = search_response.json()

  # Retrieve the paper id corresponding to the 1st result in the list
  paper_id = search_response['data'][0]['paperId']

  # Retrieve the paper details corresponding to this paper id using the function we defined earlier.
  paper_details = get_paper_data(paper_id)
  print(paper_details)
else:
  # Handle potential errors or non-200 responses
  print(f"Relevance Search Request failed with status code {search_response.status_code}: {search_response.text}")


{'paperId': 'a3121921343bef1003ff22c94c0dfc9328888375', 'title': 'Mapping the evolution of T cell states during response and resistance to adoptive cellular therapy', 'isOpenAccess': True, 'openAccessPdf': {'url': 'https://www.biorxiv.org/content/biorxiv/early/2020/07/10/2020.07.08.194332.full.pdf', 'status': 'GREEN'}}


In [63]:
if paper_details:
    if paper_details['isOpenAccess']:
        pdf_url = paper_details['openAccessPdf']['url'] 
        print(pdf_url)


https://www.biorxiv.org/content/biorxiv/early/2020/07/10/2020.07.08.194332.full.pdf


In [68]:
download_file(pdf_url, "test.pdf")

File downloaded successfully: test.pdf


# With SciHub (blocks scraping btw)

In [2]:
url = "https://sci-hub.se/10.1186/s13059-017-1382-0"
page = requests.get(url)
page

<Response [200]>

In [3]:
contents = BeautifulSoup(page.text, 'html')
contents

<!DOCTYPE html>
<html>
<head>
<title>Sci-Hub | SCANPY: large-scale single-cell gene expression data analysis. Genome Biology, 19(1) | 10.1186/s13059-017-1382-0</title>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<script src="/scripts/jquery-3.6.0.min.js"></script>
</head>
<body>
<script type="text/javascript">
        
        var menu;
        var article;
        var position;
        function rollupi()
        {
            position = position - 2;
            menu.style.left = position.toString() + 'vw';
            article.style.left = (position + 20).toString() + 'vw';
            article.style.width = (80 - position).toString() + 'vw';
            if (position > -20)
                setTimeout(rollupi, 33);
            else
                document.getElementById('rollback').style.display = 'block';
        }
        function rollup()
        {
            $.get("//sci-hub.se/rollup/on?rand=" + (Math.random() + 1).toString(36).substring(7));
    

In [24]:
pdf_tag = contents.find('embed', id="pdf")
pdf_tag.attrs

{'type': 'application/pdf',
 'src': '//zero.sci-hub.se/6995/9787dd0467a71f85a7f6dfcb2bdd1941/wolf2018.pdf#navpanes=0&view=FitH',
 'id': 'pdf'}

In [ ]:
# wget -O filename.pdf zero.sci-hub.se/6995/9787dd0467a71f85a7f6dfcb2bdd1941/wolf2018.pdf